# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [9]:
import json
import time

import pandas as pd

from bnbprob.pfssp.cython.bnb import CutoffBnB
from bnbprob.pfssp.cython.problem import (
    PermFlowShop,
    PermFlowShop1M,
    PermFlowShop2MHalf,
    PermFlowShopQuit,
)
from bnbpy import configure_logfile

In [10]:
solutions_data = """ta011\t1622\t4\t183654\t1262380\tTRUE\t1582
ta012\t1732\t3\t109364\t1022297\tTRUE\t1659
ta013\t1563\t4\t170845\t1399416\tTRUE\t1496
ta014\t1429\t1\t41096\t303451\tTRUE\t1377
ta015\t1504\t1\t39934\t317368\tTRUE\t1419
ta016\t1445\t0\t3568\t36034\tTRUE\t1397
ta017\t1539\t275\t9857028\t72692404\tTRUE\t1484
ta018\t1601\t3\t125451\t1224761\tTRUE\t1538
ta019\t1648\t0\t1667\t19578\tTRUE\t1593
ta020\t1684\t9\t290443\t2716275\tTRUE\t1591
ta021\t2394\t3600\t25248968\t199323139\tFALSE\t1996
ta022\t2181\t3600\t31692013\t262588736\tFALSE\t1930
ta023\t2386\t3600\t20696405\t179587999\tFALSE\t2008
ta024\t2262\t3600\t28491683\t238678242\tFALSE\t1985
ta025\t2353\t3600\t31450008\t271731014\tFALSE\t2048
ta026\t2283\t3600\t32208795\t270446920\tFALSE\t2067
ta027\t2386\t3600\t27428419\t251779815\tFALSE\t2067
ta028\t2283\t1546\t13474286\t118246955\tTRUE\t2200
ta029\t2360\t1486\t13348266\t114334849\tTRUE\t2237
ta030\t2260\t367\t3241893\t29204825\tTRUE\t2178
ta031\t2729\t0\t0\t1\tTRUE\t2724
ta032\t2848\t0\t10\t474\tTRUE\t2834
ta033\t2628\t0\t21\t980\tTRUE\t2621
ta034\t2773\t0\t15\t692\tTRUE\t2751
ta035\t2864\t0\t240\t10394\tTRUE\t2863
ta036\t2836\t0\t15\t709\tTRUE\t2829
ta037\t2747\t0\t7\t335\tTRUE\t2725
ta038\t2684\t0\t0\t1\tTRUE\t2683
ta039\t2590\t0\t37\t1667\tTRUE\t2552
ta040\t2782\t0\t2\t98\tTRUE\t2782
ta041\t3126\t7\t80132\t2044123\tTRUE\t2991
ta042\t3062\t405\t4132883\t98912169\tTRUE\t2867
ta043\t3025\t324\t4706113\t82527608\tTRUE\t2839
ta044\t3187\t0\t1082\t47107\tTRUE\t3063
ta045\t3146\t4\t46396\t1700215\tTRUE\t2976
ta046\t3169\t0\t77\t3487\tTRUE\t3006
ta047\t3289\t151\t1548521\t60007663\tTRUE\t3093
ta048\t3184\t0\t287\t12538\tTRUE\t3037
ta049\t3032\t3\t29661\t835678\tTRUE\t2897
ta050\t3215\t421\t5792698\t112204587\tTRUE\t3065"""

# Build the dictionary
solution_values = {}
for line in solutions_data.splitlines():
    parts = line.split('\t')
    key = parts[0]
    value = parts[-1]
    solution_values[key] = int(value)  # convert to int if you want numeric values

In [11]:
configure_logfile("pfssp-experiments.log", mode="w")

In [12]:
problems = (
    [f'ta{str(i).zfill(3)}' for i in range(11, 21, 1) if i != 17]
    # [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1) if i not in {42, 43, 47, 48, 50}]
)

In [13]:
bnbs = {}
solutions = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShopQuit.from_p(p, constructive='neh')
        bnb = CutoffBnB(
            solution_values[prob],
            eval_node='in',
            rtol=0.0001,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration, bnb.explored)

        print(f"{prob}: {sol} in {duration:.2f}s - {bnb.explored} nodes")

        bnbs[prob] = bnb
        time.sleep(0.1)

# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t, explored_nodes = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    n_nodes.append(explored_nodes)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "explored_nodes": n_nodes,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("lower-pfssp-quit-dfs.xlsx")

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 8.11s - 299650 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 6.27s - 175963 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 6.33s - 198843 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 1.44s - 48927 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 1.51s - 49231 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.15s - 3790 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 5.53s - 140221 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.10s - 1778 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 15.61s - 415331 nodes


ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 11.11s - 299673 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 8.21s - 175974 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 8.17s - 198862 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 1.78s - 48936 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 1.74s - 49235 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.17s - 3814 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 6.67s - 140232 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.12s - 1805 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 18.14s - 416667 nodes

In [14]:
bnbs = {}
solutions = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop.from_p(p, constructive='neh')
        bnb = CutoffBnB(
            solution_values[prob],
            eval_node='in',
            rtol=0.0001,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration, bnb.explored)

        print(f"{prob}: {sol} in {duration:.2f}s - {bnb.explored} nodes")

        bnbs[prob] = bnb
        time.sleep(0.1)

# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t, explored_nodes = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    n_nodes.append(explored_nodes)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "explored_nodes": n_nodes,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("lower-pfssp-2m-dfs.xlsx")

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 7.83s - 183322 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 6.32s - 108245 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 8.30s - 170608 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 1.79s - 41058 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 1.97s - 39911 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.22s - 3459 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 7.31s - 125074 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.12s - 1615 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 16.71s - 289192 nodes


ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 8.83s - 194446 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 6.76s - 127299 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 8.70s - 177677 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 2.18s - 41815 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 2.15s - 41148 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.27s - 3635 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 7.17s - 130236 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.13s - 1751 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 18.10s - 343938 nodes

In [15]:
bnbs = {}
solutions = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop1M.from_p(p, constructive='neh')
        bnb = CutoffBnB(
            solution_values[prob],
            eval_node='in',
            rtol=0.0001,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration, bnb.explored)

        print(f"{prob}: {sol} in {duration:.2f}s - {bnb.explored} nodes")

        bnbs[prob] = bnb
        time.sleep(0.1)

# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t, explored_nodes = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    n_nodes.append(explored_nodes)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "explored_nodes": n_nodes,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("lower-pfssp-1m-dfs.xlsx")

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 11.11s - 299673 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 8.21s - 175974 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 8.17s - 198862 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 1.78s - 48936 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 1.74s - 49235 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.17s - 3814 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 6.67s - 140232 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.12s - 1805 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 18.14s - 416667 nodes


In [16]:
bnbs = {}
solutions = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShop2MHalf.from_p(p, constructive='neh')
        bnb = CutoffBnB(
            solution_values[prob],
            eval_node='in',
            rtol=0.0001,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration, bnb.explored)

        print(f"{prob}: {sol} in {duration:.2f}s - {bnb.explored} nodes")

        bnbs[prob] = bnb
        time.sleep(0.1)

# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t, explored_nodes = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    n_nodes.append(explored_nodes)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "explored_nodes": n_nodes,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("lower-pfssp-2mhalf-dfs.xlsx")

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 8.83s - 194446 nodes
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 6.76s - 127299 nodes
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 8.70s - 177677 nodes
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 2.18s - 41815 nodes
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 2.15s - 41148 nodes
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 0.27s - 3635 nodes
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 7.17s - 130236 nodes
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 0.13s - 1751 nodes
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 18.10s - 343938 nodes
